### Get all the numeric features to a single csv and also add return values from CRSP

In [1]:
import pandas as pd
import numpy as np
import wrds, os
from pandas.tseries.offsets import MonthBegin

In [2]:
df = pd.read_csv('combined_train.csv')
df

,report_id,company_name,isin,reported_at,pdf_path,pages_path,blocks_path,year_month,DATE,SYMBOL,...,TICKER,SECTOR,MOMENTUM,VALUE,SIZE,BETA,SEDOL,ISIN,RETURNS,report_type
0,f_4csLsjAhudR2mcDosUbKD5,LULULEMON ATHLETICA INC,US5500211090,2024-03-21,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,2024-03,2024-03-28,55002110,...,LULU,Consumer Discretionary,44.9034,-0.609,10.840906,1.217020,B23FN39,US5500211090,-0.163652,Annual
1,f_9tgeLS94nO4peRbBEIbvje,LULULEMON ATHLETICA INC,US5500211090,2024-08-29,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,2024-08,2024-08-30,55002110,...,LULU,Consumer Discretionary,-32.0468,-0.131,10.408057,0.576961,B23FN39,US5500211090,0.003132,Quarterly
2,f_PDplDokn,LULULEMON ATHLETICA INC,US5500211090,2023-08-31,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,2023-08,2023-08-31,55002110,...,LULU,Consumer Discretionary,26.1935,-0.758,10.815705,1.478690,B23FN39,US5500211090,0.007212,Quarterly
3,f_WoQLjVEq,LULULEMON ATHLETICA INC,US5500211090,2023-12-07,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,2023-12,2023-12-29,55002110,...,LULU,Consumer Discretionary,38.3593,-0.938,11.110417,1.365380,B23FN39,US5500211090,0.144338,Quarterly
4,f_cC5GK9NDnFMWH7KTPOIlox,LULULEMON ATHLETICA INC,US5500211090,2024-12-05,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,2024-12,2024-12-31,55002110,...,LULU,Consumer Discretionary,-37.2841,-0.506,10.794169,-0.378504,B23FN39,US5500211090,0.192572,Quarterly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,o_f9saaXvCQe9wIyNvILHS3l,MOODY'S CORP,US6153691059,2023-07-06,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,2023-07,2023-07-31,61536910,...,MCO,Financials,13.1608,-0.850,11.078349,1.627940,2252058,US6153691059,0.014466,Event/Other
4242,o_fwe9aJYEVTFtxduTAdK6sd,GE VERNOVA INC,US36828A1016,2024-12-10,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,2024-12,2024-12-31,36828A10,...,GEV,Industrials,NaN,-0.803,11.419023,0.218221,BP6H4Y1,US36828A1016,-0.014815,Event/Other
4243,o_gAqntEyHWJXJDmnXkTAQEe,BANK OF AMERICA CORP,US0605051046,2024-09-30,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,2024-09,2024-09-30,06050510,...,BAC,Financials,51.9570,2.520,12.630449,1.355190,2295677,US0605051046,-0.019726,Earnings
4244,o_gdd49aPyFdkFEH34pnybsJ,CITIGROUP INC,US1729674242,2024-06-18,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,2024-06,2024-06-28,17296742,...,C,Financials,40.8893,3.993,11.673367,1.276030,2297907,US1729674242,0.018456,Event/Other


In [3]:
# Load the important page paths into a set for fast lookup
with open("pages_importance.txt", "r") as f:
    important_pages = set(line.strip() for line in f if line.strip())

# Keep only rows where df['pages_path'] is in the list
df_filtered = df[df['pages_path'].isin(important_pages)].copy()

df_filtered

,report_id,company_name,isin,reported_at,pdf_path,pages_path,blocks_path,year_month,DATE,SYMBOL,...,TICKER,SECTOR,MOMENTUM,VALUE,SIZE,BETA,SEDOL,ISIN,RETURNS,report_type
0,f_4csLsjAhudR2mcDosUbKD5,LULULEMON ATHLETICA INC,US5500211090,2024-03-21,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,2024-03,2024-03-28,55002110,...,LULU,Consumer Discretionary,44.9034,-0.609,10.840906,1.217020,B23FN39,US5500211090,-0.163652,Annual
1,f_9tgeLS94nO4peRbBEIbvje,LULULEMON ATHLETICA INC,US5500211090,2024-08-29,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,2024-08,2024-08-30,55002110,...,LULU,Consumer Discretionary,-32.0468,-0.131,10.408057,0.576961,B23FN39,US5500211090,0.003132,Quarterly
2,f_PDplDokn,LULULEMON ATHLETICA INC,US5500211090,2023-08-31,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,2023-08,2023-08-31,55002110,...,LULU,Consumer Discretionary,26.1935,-0.758,10.815705,1.478690,B23FN39,US5500211090,0.007212,Quarterly
3,f_WoQLjVEq,LULULEMON ATHLETICA INC,US5500211090,2023-12-07,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,2023-12,2023-12-29,55002110,...,LULU,Consumer Discretionary,38.3593,-0.938,11.110417,1.365380,B23FN39,US5500211090,0.144338,Quarterly
4,f_cC5GK9NDnFMWH7KTPOIlox,LULULEMON ATHLETICA INC,US5500211090,2024-12-05,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,2024-12,2024-12-31,55002110,...,LULU,Consumer Discretionary,-37.2841,-0.506,10.794169,-0.378504,B23FN39,US5500211090,0.192572,Quarterly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,o_aeN2BhuJP2gi3M2kxsHkPm,BANK OF AMERICA CORP,US0605051046,2024-06-30,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,2024-06,2024-06-28,06050510,...,BAC,Financials,43.3526,2.150,12.634439,1.420270,2295677,US0605051046,0.000499,Earnings
4239,o_biaNyxVrdz2sRiW2NToHAP,CITIGROUP INC,US1729674242,2024-07-12,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,2024-07,2024-07-31,17296742,...,C,Financials,38.8723,4.077,11.741387,1.399230,2297907,US1729674242,0.022376,Earnings
4241,o_f9saaXvCQe9wIyNvILHS3l,MOODY'S CORP,US6153691059,2023-07-06,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,2023-07,2023-07-31,61536910,...,MCO,Financials,13.1608,-0.850,11.078349,1.627940,2252058,US6153691059,0.014466,Event/Other
4243,o_gAqntEyHWJXJDmnXkTAQEe,BANK OF AMERICA CORP,US0605051046,2024-09-30,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,2024-09,2024-09-30,06050510,...,BAC,Financials,51.9570,2.520,12.630449,1.355190,2295677,US0605051046,-0.019726,Earnings


In [4]:
# Final required columns
required_cols = [
    "Sales","Expense","Profit","Ops","Liq","Inv","Fin","Lit","Emp","Tax","Acct",
    "Prod","Buyer","Process","Loc","Promo","Supplier","Mgmt","Strategy",
    "IndStruct","BizDesc","RiskFirm","RiskGeneric","Obj","Subj","Fwd","Hist",
    "analyst","Sentiment"
]

def compute_feature_avgs_and_count(pdf_path, required_cols):
    try:
        file_df = pd.read_json(pdf_path, lines=True)

        # Count rows
        row_count = len(file_df)

        # Only keep numeric cols that match required_cols
        num_cols = [c for c in required_cols if c in file_df.columns]

        avgs = {}
        if num_cols:
            avgs = file_df[num_cols].mean(skipna=True).fillna(0).to_dict()

        avgs["row_count"] = row_count
        return avgs

    except Exception as e:
        print(f"❌ Error reading {pdf_path}: {e}")
        return {col: 0 for col in required_cols + ["row_count"]}

# --- Apply to all paths in df_filtered
feature_stats = [
    compute_feature_avgs_and_count(path, required_cols)
    for path in df_filtered["pages_path"]
]

# --- Convert to DataFrame
stats_df = pd.DataFrame(feature_stats)

# --- Merge back into df_filtered
df_filtered = pd.concat(
    [df_filtered.reset_index(drop=True), stats_df.reset_index(drop=True)],
    axis=1
)

df_filtered

,report_id,company_name,isin,reported_at,pdf_path,pages_path,blocks_path,year_month,DATE,SYMBOL,...,BizDesc,RiskFirm,RiskGeneric,Obj,Subj,Fwd,Hist,analyst,Sentiment,row_count
0,f_4csLsjAhudR2mcDosUbKD5,LULULEMON ATHLETICA INC,US5500211090,2024-03-21,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,2024-03,2024-03-28,55002110,...,0.855556,1.0,0.0,0.322222,0.677778,0.622222,0.988889,0.000000,0.0,90
1,f_9tgeLS94nO4peRbBEIbvje,LULULEMON ATHLETICA INC,US5500211090,2024-08-29,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,2024-08,2024-08-30,55002110,...,0.884615,1.0,0.0,0.403846,0.596154,0.557692,0.961538,0.000000,0.0,52
2,f_PDplDokn,LULULEMON ATHLETICA INC,US5500211090,2023-08-31,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,2023-08,2023-08-31,55002110,...,0.865385,1.0,0.0,0.403846,0.596154,0.596154,0.961538,0.000000,0.0,52
3,f_WoQLjVEq,LULULEMON ATHLETICA INC,US5500211090,2023-12-07,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,2023-12,2023-12-29,55002110,...,0.854545,1.0,0.0,0.418182,0.581818,0.581818,0.963636,0.000000,0.0,55
4,f_cC5GK9NDnFMWH7KTPOIlox,LULULEMON ATHLETICA INC,US5500211090,2024-12-05,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,2024-12,2024-12-31,55002110,...,0.857143,1.0,0.0,0.392857,0.607143,0.571429,0.964286,0.000000,0.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4186,o_aeN2BhuJP2gi3M2kxsHkPm,BANK OF AMERICA CORP,US0605051046,2024-06-30,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,2024-06,2024-06-28,06050510,...,0.000000,1.0,1.0,0.150000,0.850000,0.950000,0.950000,0.000000,0.0,20
4187,o_biaNyxVrdz2sRiW2NToHAP,CITIGROUP INC,US1729674242,2024-07-12,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,2024-07,2024-07-31,17296742,...,0.105263,1.0,1.0,0.000000,1.000000,0.947368,1.000000,0.000000,0.0,19
4188,o_f9saaXvCQe9wIyNvILHS3l,MOODY'S CORP,US6153691059,2023-07-06,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,2023-07,2023-07-31,61536910,...,0.117647,1.0,1.0,0.058824,0.941176,0.941176,1.000000,0.176471,0.0,17
4189,o_gAqntEyHWJXJDmnXkTAQEe,BANK OF AMERICA CORP,US0605051046,2024-09-30,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,2024-09,2024-09-30,06050510,...,0.000000,1.0,1.0,0.062500,0.937500,0.937500,0.937500,0.000000,0.0,16


In [5]:
df_filtered.isnull().sum()

report_id       0
company_name    0
isin            0
reported_at     0
pdf_path        0
pages_path      0
blocks_path     0
year_month      0
DATE            0
SYMBOL          0
COMPANY_NAME    0
TICKER          0
SECTOR          0
MOMENTUM        0
VALUE           0
SIZE            0
BETA            0
SEDOL           0
ISIN            0
RETURNS         0
report_type     0
Sales           0
Expense         0
Profit          0
Ops             0
Liq             0
Inv             0
Fin             0
Lit             0
Emp             0
Tax             0
Acct            0
Prod            0
Buyer           0
Process         0
Loc             0
Promo           0
Supplier        0
Mgmt            0
Strategy        0
IndStruct       0
BizDesc         0
RiskFirm        0
RiskGeneric     0
Obj             0
Subj            0
Fwd             0
Hist            0
analyst         0
Sentiment       0
row_count       0
dtype: int64

In [6]:
nan_rows = df_filtered.isna().any(axis=1).sum()
print(f"Rows with at least one NaN: {nan_rows}")

Rows with at least one NaN: 0


In [7]:
df_filtered = df_filtered.dropna()

In [8]:
min_date = df_filtered["reported_at"].min()
max_date = df_filtered["reported_at"].max()

print("Min reported_at:", min_date)
print("Max reported_at:", max_date)

Min reported_at: 2023-01-04
Max reported_at: 2024-12-31


In [9]:
db = wrds.Connection()

Enter your WRDS username [komalniraula]: kn2505
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  n


You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [10]:
# Ensure unique tickers and drop NaNsn

tickers = df_filtered["TICKER"].dropna().unique().tolist()

# Format as comma-separated quoted strings for SQL
tickers_str = ",".join([f"'{t}'" for t in tickers])

# Query daily CRSP with adjustment factors
query = f"""
    select a.permno, a.date,
           a.ret, a.retx,
           a.prc, a.openprc, a.askhi, a.bidlo,
           a.vol as volume, a.shrout,
           a.cfacpr, a.cfacshr,  -- adjustment factors
           b.comnam, b.ticker
    from crsp.dsf as a
    left join crsp.stocknames as b
        on a.permno = b.permno
    where a.date between '2023-01-01' and '2024-12-30'
      and b.ticker in ({tickers_str})
"""

crsp_data = db.raw_sql(query)

# CRSP signed price convention → make absolute
for col in ["prc", "openprc", "askhi", "bidlo"]:
    crsp_data[col] = crsp_data[col].abs()

# Create adjusted prices
crsp_data["adj_close"] = crsp_data["prc"] / crsp_data["cfacpr"]
crsp_data["adj_open"]  = crsp_data["openprc"].abs() / crsp_data["cfacpr"]
crsp_data["adj_high"]  = crsp_data["askhi"].abs() / crsp_data["cfacpr"]
crsp_data["adj_low"]   = crsp_data["bidlo"].abs() / crsp_data["cfacpr"]

# Adjusted shares outstanding
crsp_data["adj_shares"] = crsp_data["shrout"] * crsp_data["cfacshr"]

# Market cap (optional, useful later)
crsp_data["mktcap"] = crsp_data["adj_close"] * crsp_data["adj_shares"]

db.close()

In [11]:
from pandas.tseries.offsets import MonthBegin

# Ensure datetime alignment
df_filtered["date"] = pd.to_datetime(df_filtered["DATE"])
crsp_data["date"] = pd.to_datetime(crsp_data["date"])

# Add empty columns
df_filtered["entry_date"] = pd.NaT
df_filtered["entry_price"] = pd.NA
df_filtered["return_after"] = pd.NA
df_filtered["return_before"] = pd.NA

# Process ticker by ticker
for ticker, reports in df_filtered.groupby("TICKER"):
    reports = reports.sort_values("date").reset_index()
    crsp_ticker = crsp_data[crsp_data["ticker"] == ticker].sort_values("date")

    for i, row in reports.iterrows():
        report_date = row["date"]

        # 1. Entry date = first trading day of next month
        start_month = (report_date + MonthBegin(1)).strftime("%Y-%m")
        entry = crsp_ticker[crsp_ticker["date"].dt.strftime("%Y-%m") == start_month].head(1)

        if entry.empty:
            continue

        entry_date = entry["date"].iloc[0]
        entry_price = entry["adj_close"].iloc[0]   # <-- use adjusted close

        # 2. Exit_after = next report’s first trading day of next month
        if i + 1 < len(reports):
            next_report = reports.loc[i + 1, "date"]
            next_month = (next_report + MonthBegin(1)).strftime("%Y-%m")
            exit_after = crsp_ticker[crsp_ticker["date"].dt.strftime("%Y-%m") == next_month].head(1)
        else:
            exit_after = pd.DataFrame()

        if not exit_after.empty:
            exit_after_price = exit_after["adj_close"].iloc[0]   # <-- adjusted
            return_after = (exit_after_price - entry_price) / entry_price
        else:
            return_after = None

        # 3. Exit_before = first trading day at least 90 days later
        exit_before = crsp_ticker[crsp_ticker["date"] >= entry_date + pd.Timedelta(days=90)].head(1)
        if not exit_before.empty:
            exit_before_price = exit_before["adj_close"].iloc[0]  # <-- adjusted
            return_before = (exit_before_price - entry_price) / entry_price
        else:
            return_before = None

        # Assign back to df_filtered
        idx = row["index"]  # from reset_index()
        df_filtered.loc[idx, "entry_date"] = entry_date
        df_filtered.loc[idx, "entry_price"] = entry_price
        df_filtered.loc[idx, "return_after"] = return_after
        df_filtered.loc[idx, "return_before"] = return_before

In [12]:
df_filtered
#return_after = after the next months report month
#return_before = after 90 days

,report_id,company_name,isin,reported_at,pdf_path,pages_path,blocks_path,year_month,DATE,SYMBOL,...,Fwd,Hist,analyst,Sentiment,row_count,date,entry_date,entry_price,return_after,return_before
0,f_4csLsjAhudR2mcDosUbKD5,LULULEMON ATHLETICA INC,US5500211090,2024-03-21,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,2024-03,2024-03-28,55002110,...,0.622222,0.988889,0.000000,0.0,90,2024-03-28,2024-04-01,385.20001,-0.215057,-0.215057
1,f_9tgeLS94nO4peRbBEIbvje,LULULEMON ATHLETICA INC,US5500211090,2024-08-29,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,2024-08,2024-08-30,55002110,...,0.557692,0.961538,0.000000,0.0,52,2024-08-30,2024-09-03,258.07999,None,0.295722
2,f_PDplDokn,LULULEMON ATHLETICA INC,US5500211090,2023-08-31,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,2023-08,2023-08-31,55002110,...,0.596154,0.961538,0.000000,0.0,52,2023-08-31,2023-09-01,404.19,0.250353,0.105421
3,f_WoQLjVEq,LULULEMON ATHLETICA INC,US5500211090,2023-12-07,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,2023-12,2023-12-29,55002110,...,0.581818,0.963636,0.000000,0.0,55,2023-12-29,2024-01-02,505.38,-0.237801,-0.237801
4,f_cC5GK9NDnFMWH7KTPOIlox,LULULEMON ATHLETICA INC,US5500211090,2024-12-05,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,2024-12,2024-12-31,55002110,...,0.571429,0.964286,0.000000,0.0,56,2024-12-31,NaT,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4186,o_aeN2BhuJP2gi3M2kxsHkPm,BANK OF AMERICA CORP,US0605051046,2024-06-30,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,2024-06,2024-06-28,06050510,...,0.950000,0.950000,0.000000,0.0,20,2024-06-28,2024-07-01,40.01,-0.012747,-0.008248
4187,o_biaNyxVrdz2sRiW2NToHAP,CITIGROUP INC,US1729674242,2024-07-12,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,2024-07,2024-07-31,17296742,...,0.947368,1.000000,0.000000,0.0,19,2024-07-31,2024-08-01,63.28,-0.028761,0.019279
4188,o_f9saaXvCQe9wIyNvILHS3l,MOODY'S CORP,US6153691059,2023-07-06,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,2023-07,2023-07-31,61536910,...,0.941176,1.000000,0.176471,0.0,17,2023-07-31,2023-08-01,352.35999,-0.112044,-0.130662
4189,o_gAqntEyHWJXJDmnXkTAQEe,BANK OF AMERICA CORP,US0605051046,2024-09-30,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,2024-09,2024-09-30,06050510,...,0.937500,0.937500,0.000000,0.0,16,2024-09-30,2024-10-01,39.22,0.064763,0.119582


In [13]:
# Select only numeric columns
num_cols = df_filtered.select_dtypes(include=[np.number])

# Compute column means
col_means = num_cols.mean()

# Check condition
all_positive = (col_means > 0).all()

print("Column means:")
print(col_means)
print("\nAll numeric columns > 0 ?", all_positive)

Column means:
MOMENTUM       11.315897
VALUE           0.580031
SIZE           10.345250
BETA            1.105478
RETURNS         0.011869
Sales           0.383864
Expense         0.326524
Profit          0.406374
Ops             0.141307
Liq             0.434752
Inv             0.171710
Fin             0.563512
Lit             0.040972
Emp             0.073163
Tax             0.222482
Acct            0.629079
Prod            0.012926
Buyer           0.005653
Process         0.006232
Loc             0.004248
Promo           0.005708
Supplier        0.024400
Mgmt            0.180249
Strategy        0.025968
IndStruct       0.056420
BizDesc         0.211872
RiskFirm        0.999940
RiskGeneric     0.980827
Obj             0.326159
Subj            0.673841
Fwd             0.601651
Hist            0.985012
analyst         0.015921
Sentiment       0.000670
row_count      57.125507
dtype: float64

All numeric columns > 0 ? True


In [14]:
df = df[df["report_type"].str.contains("Quarterly", case=False, na=False)].copy()

In [15]:
# Ensure datetime
df_filtered["reported_at"] = pd.to_datetime(df_filtered["reported_at"])

# Columns to calculate deltas
cols = [
    "Sales","Expense","Profit","Ops","Liq","Inv","Fin","Lit","Emp","Tax","Acct",
    "Prod","Buyer","Process","Loc","Promo","Supplier","Mgmt","Strategy",
    "IndStruct","BizDesc","RiskFirm","RiskGeneric","Obj","Subj","Fwd","Hist",
    "analyst","Sentiment"
]

# Compute deltas per company (group by isin)
df_filtered = df_filtered.sort_values(["isin","reported_at"])

for col in cols:
    delta_col = f"delta_{col}_avg"

    df_filtered[delta_col] = df_filtered[col] - df_filtered.groupby("isin")[col].shift(1)

In [16]:
df_filtered.to_csv('final_data.csv', index=False)